# ENCRYPT - FEITO

In [358]:
import os
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes, hmac
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from pickle import dumps

USED_NONCES = []

BLOCK_SIZE = 16

def generate_mac(key, crypto):
    h = hmac.HMAC(key, hashes.SHA256(), backend = default_backend())
    h.update(crypto)
    return h.finalize()

def split_into_blocks(plaintext, block_size):
    return [plaintext[i:i+block_size] for i in range(0, len(plaintext), block_size)]

def gen_nonce():
    nonce = os.urandom(BLOCK_SIZE // 2)       # 128/2 bit = 64 bit = 8 byte
    while nonce in USED_NONCES:
        nonce = os.urandom(BLOCK_SIZE // 2)   # 128/2 bit = 64 bit = 8 byte
    USED_NONCES.append(nonce)
    return nonce

# i is the tweak number
def gen_tweak(i, plaintext_len=''):
    
    if i > 0:
        nonce = gen_nonce() # Generates nonce with 8 byte
        # Concate between nonce (8 byte) with i (8 byte) [N||i]
        tweak = nonce + int(i).to_bytes(BLOCK_SIZE // 2, byteorder='big') 
        tweak = int.from_bytes(tweak, byteorder='big') # Convert to int
        tweak = tweak >> 1 # Removes last bit
        tweak = tweak << 1 # Adds 0 bit in the last position [N||i||0]
        tweak = tweak.to_bytes(BLOCK_SIZE, byteorder='big')
    else:
        nonce = gen_nonce() # Generates nonce with 8 byte
        # Concate between nonce (8 byte) with plaintext_len (8 byte) [N||b]
        tweak = nonce + int(plaintext_len).to_bytes(BLOCK_SIZE // 2, byteorder='big')
        # Adds 1 bit in the last position [N||b||1]
        tweak = tweak[:-1] + (tweak[-1] | ord('\x01')).to_bytes(1,byteorder='big')
    
    return tweak  

def bytes_xor(bytes1, bytes2):
    return b''.join([(a^b).to_bytes(1,byteorder='big') for (a,b) in zip(bytes1, bytes2)])

def encrypt(key, tweak, block):
    iv1 = os.urandom(16)
    iv2 = os.urandom(16)
    
    encryptor = Cipher(algorithms.AES(key), 
                       modes.CBC(iv1),
                       backend=default_backend()).encryptor()
    crypto = encryptor.update(block) + encryptor.finalize()
    
    crypto_xor_tweak = bytes_xor(crypto, tweak)
    
    encryptor = Cipher(algorithms.AES(key), 
                       modes.CBC(iv2),
                       backend=default_backend()).encryptor()
    crypto = encryptor.update(crypto_xor_tweak) + encryptor.finalize()
    
    return crypto, iv1, iv2
    
    

def tae_encrypt(plaintext, key):    
    blocks = split_into_blocks(plaintext.encode(), BLOCK_SIZE) # Blocks size: 16 byte -> 128 bit
    last_block_len = len(blocks[-1]).to_bytes(BLOCK_SIZE, byteorder='big') # Length of last block
    blocks[-1] = blocks[-1] + b'0' * (BLOCK_SIZE - len(blocks[-1])) # Last block padded with zeros
    
    cryptos = []
    tweaks = []
    ivs = []
    
    # Authentication tag
    tweak_0 = gen_tweak(0, len(plaintext))
    tweaks.append(tweak_0)
    checksum = blocks[0]
    for b in blocks[1:]:
        checksum = bytes_xor(checksum, b)

    tag, iv1, iv2 = encrypt(key[:32], tweak_0, checksum)
    ivs.append((iv1, iv2))
    tag = tag[:BLOCK_SIZE]
    
    for i,block in enumerate(blocks, start=1):
        tweak_i = gen_tweak(i)
        tweaks.append(tweak_i)
        
        if i != len(blocks):
            crypto, iv1, iv2 = encrypt(key[:32], tweak_i, block)
            ivs.append((iv1, iv2))
            cryptos.append(crypto)
            
        else: # Last block
            # Produces a n-bit binary representation of the length r
            last_crypto, iv1, iv2 = encrypt(key[:32], tweak_i, last_block_len)
            ivs.append((iv1, iv2))
            # CRYPTO XOR M
            cryptos.append(bytes_xor(last_crypto, block))
            
    ciphertext = b''.join(cryptos)

    package = { 'ivs': ivs, 'tag': tag, 'crypto': ciphertext, 'tweaks': tweaks } # Mais alguma coisa? Tweaks? Nonce?

    hmac = generate_mac(key[32:64], dumps(package))

    return {'mess' : package, 'tag' : hmac}

# DECRYPT - AQUI FALTA FAZER PARA O ÚLTIMO BLOCO

In [365]:
def decrypt(key, iv1, iv2, tweak, ciphertext):
    decryptor = Cipher(algorithms.AES(key), 
                       modes.CBC(iv2),
                       backend=default_backend()).decryptor()
    crypto = decryptor.update(ciphertext) + decryptor.finalize()
    
    crypto_xor_tweak = bytes_xor(crypto, tweak)
    
    decryptor = Cipher(algorithms.AES(key), 
                       modes.CBC(iv1),
                       backend=default_backend()).decryptor()
    plaintext = decryptor.update(crypto_xor_tweak) + decryptor.finalize()
    return plaintext
    

def tae_decrypt(pkg, key):
    
    crypto = pkg['mess']
    hmac = pkg['tag']
    
    macDest = generate_mac(key[32:64], dumps(crypto))
    if (hmac != macDest):
        return 'ERROR - MAC/Password is not equal'
    
    tag = crypto['tag']
    ciphertext = crypto['crypto']
    tweaks = crypto['tweaks']
    ivs = crypto['ivs']
    
    blocks = split_into_blocks(ciphertext, BLOCK_SIZE) # Blocks size: 16 byte -> 128 bit
    
    plaintexts = []
    
    for i,block in enumerate(blocks, start=1):
        tweak_i = tweaks[i]
        iv1 = ivs[i][0]
        iv2 = ivs[i][1]
        
        if i != len(blocks):
            plaintexts.append(decrypt(key[:32], iv1, iv2, tweak_i, block))
            
        else: # Last block
            pass # <---------------------------------------------------------------------
        
            
    # Authentication tag
    checksum = plaintexts[0]
    for pt in plaintexts[1:]:
        checksum = bytes_xor(checksum, pt)
        
    new_tag = decrypt(key[:32], ivs[0][0], ivs[0][1], tweaks[0], checksum)
    new_tag = new_tag[:BLOCK_SIZE]
    
    if new_tag != tag:
        return 'ERROR - TAG is not equal'

    return (b''.join(plaintexts)).decode()

key = os.urandom(64)
tae_decrypt(tae_encrypt('isto e uma mensagem cumprida que espero que seja separada em varios blocos e cada um tenha 128 bits', key), key) 

'ERROR - TAG is not equal'

# DAQUI PARA BAIXO NÃO ESTÁ NADA FEITO

In [4]:
from multiprocessing import Process, Pipe

def Alice(conn):
    conn.send('hello')  # envia uma mensagem pelo seu lado do Pipe
    conn.close()        # termina a ligação do seu lado do Pipe
    
def Bob(conn):
    mess = conn.recv()  # recebe a mensagem do seu lado do Pipe
    print(mess)         # faz qualquer coisa com a informação recebida
    conn.close()        # fecha a ligação do seu lado
    
class BiConn(object):
    def __init__(self,left,right,timeout=None):
        """
        left : a função que vei ligar ao lado esquerdo do Pipe
        right: a função que vai ligar ao outro lado
        timeout: (opcional) numero de segundos que aguarda pela terminação do processo
        """
        left_end, right_end = Pipe()
        self.timeout=timeout
        self.lproc = Process(target=left, args=(left_end,))       # os processos ligados ao Pipe
        self.rproc = Process(target=right, args=(right_end,))
        self.left  = lambda : left(left_end)                       # as funções ligadas já ao Pipe
        self.right = lambda : right(right_end)

    def auto(self, proc=None):
        if proc == None:             # corre os dois processos independentes
            self.lproc.start()
            self.rproc.start()  
            self.lproc.join(self.timeout)
            self.lproc.join(self.timeout)
        else:                        # corre só o processo passado como parâmetro
            proc.start(); proc.join()

    def manual(self):   #  corre as duas funções no contexto de um mesmo processo Python
        self.left()
        self.right()
    
        
Conn = BiConn(Alice,Bob)
Conn.auto()

hello
